In [68]:

import tensorflow as tf
from matplotlib import pyplot
from keras.datasets import cifar10
# from emnist import extract_training_samples
from tensorflow.keras import *
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG19
import random

## Vanilla VGG 

In [98]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

Y_train = y_train
Y_test = y_test


num_classes = 10 
x_train = x_train/255.
y_train = utils.to_categorical(y_train, num_classes) 

x_test = x_test/255.
y_test = utils.to_categorical(y_test, num_classes) 



In [3]:
model3 = tf.keras.models.load_model("saved-models/cifar10_vgg_200ep/model.h5")


2022-01-01 17:51:04.385730: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-01 17:51:04.385770: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-01 17:51:04.385790: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (x-dell): /proc/driver/nvidia/version does not exist
2022-01-01 17:51:04.386023: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
model3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 1, 1, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 10)                5130      
                                                                 
Total params: 20,029,514
Trainable params: 20,029,514
Non-trainable params: 0
_________________________________________________________________


In [6]:
model3.evaluate(x_test, y_test)

2022-01-01 17:54:17.187490: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 122880000 exceeds 10% of free system memory.


313/313 [==============================] - 42s 133ms/step - loss: 3.1982 - accuracy: 0.7218


[3.198240280151367, 0.7218000292778015]

# Siamese VGG

In [126]:
model = tf.keras.models.load_model("saved-models/cifar10_vgg_siamese-crossentropy/model.h5")

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 vgg19A (Functional)            (None, 1, 1, 512)    20024384    ['input_1[0][0]']                
                                                                                                  
 vgg19B (Functional)            (None, 1, 1, 512)    20024384    ['input_2[0][0]']                
                                                                                              

In [127]:
def make_pairs(images, labels):
    pairImages = []
    pairLabels = []

    numClasses = len(np.unique(labels))
    idx = [np.where(labels == i)[0] for i in range(0, numClasses)]

    for idxA in range(len(images)):
        currentImage = images[idxA]
        label = labels[idxA]
        
        for idxB in random.sample(list(idx[label[0]]), 1):
            posImage = images[idxB]

            pairImages.append(np.array([currentImage, posImage]))
            pairLabels.append(np.array(label))
        
    return np.array(pairImages), np.array(pairLabels)


x_train_siamese, y_train_siamese = make_pairs(x_train, Y_train)
x_test_siamese, y_test_siamese = make_pairs(x_test, Y_test)

y_train_siamese = utils.to_categorical(y_train_siamese, num_classes) 
y_test_siamese = utils.to_categorical(y_test_siamese, num_classes) 



x_train_both = [x_train_siamese[:,0,:], x_train_siamese[:,1,:]]
x_test_both = [x_test_siamese[:,0,:], x_test_siamese[:,1,:]]

In [129]:
model.evaluate(x_test_both, y_test_siamese)

313/313 [==============================] - 110s 340ms/step - loss: 2.3026 - accuracy: 0.0987


[2.3025856018066406, 0.09870000183582306]